<a href="https://colab.research.google.com/github/VISHAL2077/Generative_AI_2025/blob/main/GenAI_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input
from tqdm import tqdm
import cv2

IMAGE_FOLDER = '/content/drive/MyDrive/ClothingAttributeDataset'
OUTPUT_CSV = '/content/drive/MyDrive/ClothingAttributeDataset/image_features_with_labels.csv'

model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

def extract_features(img_path):
    try:
        img = image.load_img(img_path, target_size=(224, 224))
        img_data = image.img_to_array(img)
        img_data = np.expand_dims(img_data, axis=0)
        img_data = preprocess_input(img_data)
        features = model.predict(img_data)
        return features.flatten()
    except Exception as e:
        return None

features_list = []
image_names = []

print("✅ Extracting features from all subfolders...")

for root, dirs, files in os.walk(IMAGE_FOLDER):
    for img_name in files:
        if img_name.endswith('.jpg') or img_name.endswith('.png'):
            img_path = os.path.join(root, img_name)
            features = extract_features(img_path)
            if features is not None:
                features_list.append(features)
                image_names.append(img_name)
            else:
                print(f"⚠️ Skipped corrupted image: {img_name}")

if len(features_list) == 0:
    print("❌ No valid images found. Please check the dataset.")
    exit()

df = pd.DataFrame(features_list)
df.insert(0, 'Image', image_names)

scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(df.drop('Image', axis=1))

pca = PCA(n_components=10)
features_reduced = pca.fit_transform(features_scaled)

kmeans = KMeans(n_clusters=10, random_state=42)
clusters = kmeans.fit_predict(features_reduced)

costume_map = {
    0: ['T-Shirt', 'Both'],
    1: ['Jeans', 'Both'],
    2: ['Dress', 'Women Only'],
    3: ['Blazer', 'Men Only'],
    4: ['Frock', 'Women Only'],
    5: ['Coat', 'Both'],
    6: ['Skirt', 'Women Only'],
    7: ['Formal Pants', 'Men Only'],
    8: ['Hoodie', 'Both'],
    9: ['Gown', 'Women Only'],
    10: ['Jumpsuit', 'Women Only'],
    11: ['Shorts', 'Both'],
    12: ['Suit', 'Men Only'],
    13: ['Sweatshirt', 'Both'],
    14: ['Shirt', 'Men Only']
}

categories = [costume_map[label] for label in clusters]

category_df = pd.DataFrame(categories, columns=['Costume Name', 'Wearable By'])

final_df = pd.concat([df['Image'], category_df], axis=1)

final_df.to_csv(OUTPUT_CSV, index=False)

print(f"✅ Features and categories (with Wearable By) extracted. CSV saved at: {OUTPUT_CSV}")
print("✅ Finished successfully!")



✅ Extracting features from all subfolders...
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 209ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 204ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 210ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 210ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 211ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 211ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 310ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 352ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 287ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 207ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 214ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 218ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 210ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 218ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 234ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 506ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 227ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 254ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 209ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 209ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step
1/1 ━━━━━━━━━━━━━━━━━━

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Reshape, LeakyReLU, BatchNormalization, Embedding, Flatten
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Concatenate
from sklearn.preprocessing import LabelEncoder
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd

# ✅ Load your dataset
df = pd.read_csv('/content/drive/MyDrive/ClothingAttributeDataset/image_features_with_labels.csv')

# ✅ Encode the text labels
le = LabelEncoder()
df['Costume Name'] = le.fit_transform(df['Costume Name'])
df['Wearable By'] = le.fit_transform(df['Wearable By'])

# ✅ Generator Model
def build_generator(latent_dim, num_classes):
    noise_input = Input(shape=(latent_dim,))
    label_input = Input(shape=(1,))

    # Embed the label
    label_embedding = Embedding(num_classes, 50)(label_input)
    label_embedding = Flatten()(label_embedding)

    # Combine noise + label
    combined_input = Concatenate()([noise_input, label_embedding])

    # Build Generator
    model = Sequential()
    model.add(Dense(256, input_dim=latent_dim+50))
    model.add(LeakyReLU(0.2))
    model.add(Dense(512))
    model.add(LeakyReLU(0.2))
    model.add(Dense(1024))
    model.add(LeakyReLU(0.2))
    model.add(Dense(28*28*3, activation='tanh'))
    model.add(Reshape((28,28,3)))

    output = model(combined_input)
    return Model([noise_input, label_input], output)

# ✅ Discriminator Model
def build_discriminator(num_classes):
    image_input = Input(shape=(28,28,3))
    label_input = Input(shape=(1,))

    # Embed the label
    label_embedding = Embedding(num_classes, 50)(label_input)
    label_embedding = Flatten()(label_embedding)

    # Combine image + label
    combined_input = Concatenate()([Flatten()(image_input), label_embedding])

    # Build Discriminator
    model = Sequential()
    model.add(Dense(1024))
    model.add(LeakyReLU(0.2))
    model.add(Dense(512))
    model.add(LeakyReLU(0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(0.2))
    model.add(Dense(1, activation='sigmoid'))

    output = model(combined_input)
    return Model([image_input, label_input], output)

# ✅ Compile the GAN
latent_dim = 100
num_classes = len(df['Costume Name'].unique())

generator = build_generator(latent_dim, num_classes)
discriminator = build_discriminator(num_classes)

discriminator.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
discriminator.trainable = False

gan_input_noise = Input(shape=(latent_dim,))
gan_input_label = Input(shape=(1,))
gan_output = discriminator([generator([gan_input_noise, gan_input_label]), gan_input_label])

gan = Model([gan_input_noise, gan_input_label], gan_output)
gan.compile(loss='binary_crossentropy', optimizer='adam')

# ✅ Train the CGAN
def train(epochs, batch_size):
    X_train = np.random.normal(0, 1, (len(df), latent_dim))
    y_train = df['Costume Name'].values

    for epoch in range(epochs):
        # Generate fake images
        random_noise = np.random.normal(0, 1, (batch_size, latent_dim))
        random_labels = np.random.randint(0, num_classes, batch_size)
        generated_images = generator.predict([random_noise, random_labels])

        # Real images
        real_images = np.random.normal(0, 1, (batch_size, 28, 28, 3))
        real_labels = np.ones((batch_size, 1))

        # Fake labels
        fake_labels = np.zeros((batch_size, 1))

        # Train discriminator
        d_loss_real = discriminator.train_on_batch([real_images, random_labels], real_labels)
        d_loss_fake = discriminator.train_on_batch([generated_images, random_labels], fake_labels)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train generator
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        labels = np.random.randint(0, num_classes, batch_size)
        g_loss = gan.train_on_batch([noise, labels], real_labels)

        print(f"Epoch {epoch}/{epochs} | D Loss: {d_loss} | G Loss: {g_loss}")

# ✅ Train the model
train(epochs=5000, batch_size=64)

# ✅ Generate Image Function
def generate_image(text_input):
    text_to_label = le.transform([text_input])[0]
    noise = np.random.normal(0, 1, (1, latent_dim))
    generated_image = generator.predict([noise, np.array([text_to_label])])

    # Show Image
    plt.imshow((generated_image[0] * 127.5 + 127.5).astype(np.uint8))
    plt.axis('off')
    plt.show()

# ✅ Generate Test Images
while True:
    user_text = input("Enter clothing preference (type 'exit' to stop): ")
    if user_text.lower() == 'exit':
        break
    generate_image(user_text)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step 


/usr/local/lib/python3.11/dist-packages/keras/src/backend/tensorflow/trainer.py:82: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


Epoch 0/5000 | D Loss: [0.7260819  0.47265625] | G Loss: 0.6901026964187622
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
Epoch 1/5000 | D Loss: [0.9354033 0.4329427] | G Loss: 0.3915565609931946
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Epoch 2/5000 | D Loss: [1.4285035  0.38385415] | G Loss: 0.2625828683376312
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Epoch 3/5000 | D Loss: [2.1098976 0.3515625] | G Loss: 0.19698451459407806
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Epoch 4/5000 | D Loss: [2.845049   0.33151042] | G Loss: 0.15758933126926422
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Epoch 5/5000 | D Loss: [3.5678282  0.31853694] | G Loss: 0.1313246339559555
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Epoch 6/5000 | D Loss: [4.246191   0.31988323] | G Loss: 0.11256398260593414
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Epoch 7/5000 | D Loss: [4.872478   0.31484374] | G Loss: 0.09849348664283752
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Epoch 8/5000 | D Loss: [5.448049   0.31096816] | G Loss: 0.08754976838827133
2/2 ━━━━━━